Diabetes prediction with classification type of knn without using sklrean for the knn part to further understand the concept.

In [92]:
import pandas as pd
import numpy as np

getting information:

In [3]:
df = pd.read_csv("diabetes.csv")
print(df.head(5))

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [13]:
print(df.columns)
print(df.shape)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')
(768, 9)


In [6]:
print(df.isna().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [10]:
print(df.describe())

       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.244002   
min       0.000000    0.000000       0.000000       0.000000    0.000000   
25%       1.000000   99.000000      62.000000       0.000000    0.000000   
50%       3.000000  117.000000      72.000000      23.000000   30.500000   
75%       6.000000  140.250000      80.000000      32.000000  127.250000   
max      17.000000  199.000000     122.000000      99.000000  846.000000   

              BMI  DiabetesPedigreeFunction         Age     Outcome  
count  768.000000                768.000000  768.000000  768.000000  
mean    31.992578                  0.471876   33.240885    0.348958  
std      7.884160                  0.331329   11.760232    0.476951  
min      0.000000                  

In [12]:
col = df.columns
zero_counts = (df[col]==0).sum()
print(zero_counts) 

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64


Data Cleaning ---> Deciding on Zero Values

In [15]:
df["Glucose"] = df["Glucose"].replace(0 , df["Glucose"].mean())

In [18]:
df["BloodPressure"] = df["BloodPressure"].replace(0 , df["BloodPressure"].mean())

In [19]:
df["BMI"] = df["BMI"].replace(0 , df["BMI"].mean())

In [20]:
df["Insulin"] = df["Insulin"].replace(0 , df["Insulin"].median())

In [21]:
df["SkinThickness"] = df["SkinThickness"].replace(0 , df["SkinThickness"].median())

In [22]:
col = df.columns
zero_counts = (df[col]==0).sum()
print(zero_counts) 

Pregnancies                 111
Glucose                       0
BloodPressure                 0
SkinThickness                 0
Insulin                       0
BMI                           0
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64


determining x and y ---> 
SHUFFLLE is needed in here cause what if the data is organised from healthy to diabetes???
then my test part will be useless.

In [56]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [57]:
x = df.drop("Outcome" , axis=1)
print(x.head(5))

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6    102.0           90.0             39     30.5  35.7   
1            5    109.0           75.0             26     30.5  36.0   
2            2    122.0           70.0             27     30.5  36.8   
3            4     76.0           62.0             23     30.5  34.0   
4            7    150.0           66.0             42    342.0  34.7   

   DiabetesPedigreeFunction  Age  
0                     0.674   28  
1                     0.546   60  
2                     0.340   27  
3                     0.391   25  
4                     0.718   42  


In [58]:
print(x.shape)
print((x.columns==0).sum())
print(x.isna().sum())

(768, 8)
0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64


In [59]:
y = df["Outcome"]
print(y.shape)

(768,)


In [60]:
print((y==0).sum())
print(y.isna().sum())

500
0


x_train , y_train , x_test , y_test

In [61]:
num_size = int(0.8*len(x)) #as long as i may change something in this data in the future, i need something constant.
x_train = x[:num_size]
y_train = y[:num_size]
x_test = x[num_size:]
y_test = y[num_size:]

scaling x ---> cause knn is sensitive about distance 

In [62]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)     
x_train_scaled = scaler.transform(x_train) 
x_test_scaled = scaler.transform(x_test)    

KNN

In [88]:
y_pred = []

for test in x_test_scaled:

    distance = np.sum((x_train_scaled - test)**2, axis=1)

    nearest_index_x = np.argsort(distance)[:8]
    y_value = y_train.iloc[nearest_index_x]
    nearest_y = y_value.value_counts().idxmax()
    y_pred.append(nearest_y)
y_pred = np.array(y_pred)
print(y_pred)

[1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0
 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 0
 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 0 0
 0 1 1 1 0 1]


accuracy:

In [89]:
print(x_test.iloc[2])
print(y_test.iloc[2])
print(y_pred[2])



Pregnancies                   6.000
Glucose                     108.000
BloodPressure                44.000
SkinThickness                20.000
Insulin                     130.000
BMI                          24.000
DiabetesPedigreeFunction      0.813
Age                          35.000
Name: 616, dtype: float64
0
0


In [91]:
correct = np.sum(y_pred == y_test.values)
total = len(y_test)
accuracy = correct / total
print("Accuracy:", accuracy)
accuracy = np.mean(y_pred == y_test.values)
print(f"Accuracy: {accuracy:.2f}")

# more that 60 precent accuracy is fine

Accuracy: 0.7207792207792207
Accuracy: 0.72
